In [133]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize
from torch.utils.data import Subset

In [134]:
transform = torchvision.transforms.Compose([ToTensor(), Normalize((0.5), (0.5))])

train_dataset = datasets.FashionMNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST('./data', train=False, transform=transform)

In [135]:
# Splitting datasets for two tasks
train_dataset_task1 = Subset(train_dataset, [i for i in range(len(train_dataset)) if train_dataset.targets[i] <= 5])
train_dataset_task2 = Subset(train_dataset, [i for i in range(len(train_dataset)) if train_dataset.targets[i] > 5])
test_dataset_task1 = Subset(test_dataset, [i for i in range(len(test_dataset)) if test_dataset.targets[i] <= 5])
test_dataset_task2 = Subset(test_dataset, [i for i in range(len(test_dataset)) if test_dataset.targets[i] > 5])

In [136]:
unique_labels_1 = set()
unique_labels_2 = set()

for _, target in train_dataset_task1:
    unique_labels_1.add(target)

for _, target in train_dataset_task2:
    unique_labels_2.add(target)

print(f"First: {unique_labels_1}")
print(f"Second: {unique_labels_2}")

First: {0, 1, 2, 3, 4, 5}
Second: {8, 9, 6, 7}


In [137]:
train_dataloader_task1 = DataLoader(train_dataset_task1, batch_size=64, shuffle=True)
test_dataloader_task1 = DataLoader(test_dataset_task1, batch_size=256, shuffle=True)

for X, y in train_dataloader_task1:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"SHape of y: {y.shape}, dtype: {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
SHape of y: torch.Size([64]), dtype: torch.int64


In [138]:
train_dataloader_task2 = DataLoader(train_dataset_task2, batch_size=64, shuffle=True)
test_dataloader_task2 = DataLoader(test_dataset_task2, batch_size=256, shuffle=True)

In [139]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cuda device


In [140]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes=10, hidden_size=512):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(28*28, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.classifier = nn.Linear(hidden_size, num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, nonlinearity='sigmoid')
            elif isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        logits = self.classifier(x)

        return logits

In [141]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")

In [142]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg Loss: {test_loss:>8f}\n")

In [143]:
model_task1 = NeuralNetwork(num_classes=10, hidden_size=512).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_task1.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [144]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n---------------------------")
  train(train_dataloader_task1, model_task1, loss_fn, optimizer)
  test(test_dataloader_task1, model_task1, loss_fn)
print("Done!")

Epoch 1
---------------------------
Loss: 2.377352,    64/36000
Loss: 0.280044,  6464/36000
Loss: 0.205652, 12864/36000
Loss: 0.361546, 19264/36000
Loss: 0.295533, 25664/36000
Loss: 0.332733, 32064/36000
Test Error: 
 Accuracy: 90.2, Avg Loss: 0.268524

Epoch 2
---------------------------
Loss: 0.187860,    64/36000
Loss: 0.170237,  6464/36000
Loss: 0.270315, 12864/36000
Loss: 0.244447, 19264/36000
Loss: 0.272340, 25664/36000
Loss: 0.119911, 32064/36000
Test Error: 
 Accuracy: 88.3, Avg Loss: 0.311424

Epoch 3
---------------------------
Loss: 0.242517,    64/36000
Loss: 0.158272,  6464/36000
Loss: 0.139934, 12864/36000
Loss: 0.169928, 19264/36000
Loss: 0.278709, 25664/36000
Loss: 0.258056, 32064/36000
Test Error: 
 Accuracy: 91.4, Avg Loss: 0.241399

Epoch 4
---------------------------
Loss: 0.165632,    64/36000
Loss: 0.072139,  6464/36000
Loss: 0.289277, 12864/36000
Loss: 0.249114, 19264/36000
Loss: 0.228521, 25664/36000
Loss: 0.108559, 32064/36000
Test Error: 
 Accuracy: 91.3, Avg 

In [145]:
def val(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()

    correct = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}\n")

In [146]:
val(test_dataloader_task1, model_task1)

Test Error: 
 Accuracy: 90.6



________

In [147]:
# model = NeuralNetwork()
# model.load_state_dict(torch.load("model_old.pth"))

In [148]:
from copy import deepcopy

def get_fisher_diag(model, dataloader, params, empirical=False):
    fisher = {}
    params_dict = dict(params)
    for n, p in deepcopy(params_dict).items():
        p.data.zero_()
        fisher[n] = p.data.clone().detach().requires_grad_()

    model.eval()

    for input, gt_label in dataloader:
        input, gt_label = input.to(device), gt_label.to(device)
        model.zero_grad()
        output = model(input)

        if empirical:
            label = output.max(1)[1]
        else:
            label = gt_label

        # label = gt_label.repeat(output.size(0))
        negloglikelihood = torch.nn.functional.nll_loss(torch.nn.functional.log_softmax(output, dim=1), label)
        negloglikelihood.backward()

        for n, p in model.named_parameters():
            fisher[n].data += p.grad.data ** 2 / len(dataloader.dataset)

    fisher = {n: p for n, p in fisher.items()}
    return fisher


def get_ewc_loss(model, fisher, p_old):
    loss = 0
    for n, p in model.named_parameters():
        _loss = fisher[n] * (p - p_old[n]) ** 2
        loss += _loss.sum()
    return loss

In [149]:
model_task2 = model_task1

In [150]:
model_task2.to(device)

ewc_lambda = 1_000_000

fisher_matrix = get_fisher_diag(model_task2, train_dataloader_task1, model_task2.named_parameters())
prev_params = {n: p.data.clone() for n, p in model_task2.named_parameters()}

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_task2.parameters(), lr=0.001, momentum=0.9)

In [151]:
def train(dataloader, model, loss_fn, optimizer, fisher_matrix, prev_params):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)

        # Original loss
        ce_loss = loss_fn(pred, y)

        # EWC loss
        ewc_loss = get_ewc_loss(model, fisher_matrix, prev_params)

        loss = ce_loss + ewc_lambda * ewc_loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"Loss: {loss:>7f}, {current:>5d}/{size:>5d}")

In [152]:
# def val(epoch):
#     model.eval()
#     correct, total = 0, 0
#     with torch.no_grad():
#         for batch, (X, y) in enumerate(test_dataloader_task1):
#             X, y = X.to(device), y.to(device)
#             outputs = model(X)
#             _, predicted_old = outputs.max(1)
#             print(torch.unique(predicted_old))
#             total += len(y)
#             correct += predicted_old.eq(y).sum().item()
#         print(f"Validation Acc: {100. * correct / total}\n")

def val(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()

    correct = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            _, predicted_old = pred.max(1)
            print(torch.unique(predicted_old))
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    correct /= size
    print(f"Validation Accuracy: {(100*correct):>0.1f}\n")

In [153]:
for epoch in range(5):
    print(f"Epoch {epoch+1}: ----------------------")
    train(train_dataloader_task2, model_task2, loss_fn, optimizer, fisher_matrix, prev_params)
    test(test_dataloader_task2, model_task2, loss_fn)
print("Done!")

Epoch 1: ----------------------
Loss: 12.273902,    64/24000
Loss: 0.599661,  6464/24000
Loss: 0.342755, 12864/24000
Loss: 0.293038, 19264/24000
Test Error: 
 Accuracy: 94.1, Avg Loss: 0.178757

Epoch 2: ----------------------
Loss: 0.315878,    64/24000
Loss: 0.178056,  6464/24000
Loss: 0.193526, 12864/24000
Loss: 0.187861, 19264/24000
Test Error: 
 Accuracy: 94.9, Avg Loss: 0.158991

Epoch 3: ----------------------
Loss: 0.153440,    64/24000
Loss: 0.208027,  6464/24000
Loss: 0.126336, 12864/24000
Loss: 0.216780, 19264/24000
Test Error: 
 Accuracy: 95.3, Avg Loss: 0.140433

Epoch 4: ----------------------
Loss: 0.289167,    64/24000
Loss: 0.187533,  6464/24000
Loss: 0.207336, 12864/24000
Loss: 0.167752, 19264/24000
Test Error: 
 Accuracy: 95.9, Avg Loss: 0.129740

Epoch 5: ----------------------
Loss: 0.291168,    64/24000
Loss: 0.150990,  6464/24000
Loss: 0.094746, 12864/24000
Loss: 0.128864, 19264/24000
Test Error: 
 Accuracy: 96.0, Avg Loss: 0.121620

Done!


In [154]:
val(test_dataloader_task1, model_task2)

tensor([0, 1, 3, 4, 6, 7, 8, 9], device='cuda:0')
tensor([0, 1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([0, 1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([0, 1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([0, 1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([0, 1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([0, 1, 3, 6, 7, 8, 9], device='cuda:0')
tensor([1, 2, 3, 6, 7, 8, 9], device='cuda:0')
tensor([0, 1, 3, 6, 7, 8, 9], device='cuda:0')
tenso